In [380]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data_path = "./data/"
df = pd.read_csv(data_path + "teste_indicium_precificacao.csv")


In [381]:
df[['bairro', 'bairro_group']]

,bairro,bairro_group
0,Midtown,Manhattan
1,Harlem,Manhattan
2,Clinton Hill,Brooklyn
3,East Harlem,Manhattan
4,Murray Hill,Manhattan
...,...,...
48889,Bedford-Stuyvesant,Brooklyn
48890,Bushwick,Brooklyn
48891,Harlem,Manhattan
48892,Hell's Kitchen,Manhattan


In [382]:
# Criando um conjunto de treino
# Aqui, estamos criando conjunto de treino, teste e validação de forma estratificada
# Primeiro, separamos o conjunto de treino com 80% dos dados e de teste com 20%
# Depois, o conjunto de validação terá 12,5% do conjunto de treino
# Porcentagens finais: Treino: (87.5 * 80)/100 = 70%, teste 20% e validação: (12.5 * 80)/100 = 10%
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, stratify=df['bairro_group'], random_state=42)
train_set, val_set = train_test_split(train_set, test_size=0.125, stratify=train_set['bairro_group'], random_state=42)

In [383]:
housing = train_set.drop("price", axis=1)
housing_labels = train_set['price'].copy()


In [384]:
def custom_preprocess(df):
    # Primeiro, ao meu julgamento, as colunas "id", "nome", "host_id", "host_name" e  "ultima_review" não influenciam na previsão
    def remove_irrelevant(df, columns):
        return df.drop(columns=columns)
    def central_park_proximity(df):
        central_park_geo = {
            'latitude_min': 40.758112,
            'latitude_max': 40.807937,
            'longitude_min': -74,
            'longitude_max': -73.968656 
        }
        def prox_centralpark(row):
            cond1 = row['latitude'] >= central_park_geo['latitude_min'] and row['latitude'] <= central_park_geo['latitude_max']
            cond2 = row['longitude'] >= central_park_geo['longitude_min'] and row['longitude'] <= central_park_geo['longitude_max']
            if cond1 and cond2: 
                return 1
            return 0

        df['near_central_park'] = df.apply(prox_centralpark, axis=1)
        return df

    df = remove_irrelevant(df, ['id', 'nome', 'host_id', 'host_name', 'ultima_review'])
    df = central_park_proximity(df)
    return df

In [385]:
housing = custom_preprocess(housing)

In [386]:
# Preparar os dados para encoding
housing_num = housing.drop(columns=['bairro_group', 'bairro', 'room_type'])
num_attrs = list(housing_num)
# Especificamente para esse problema, usarei OrdinalEncoder para bairro_group e room_type e OneHotEncoder para bairro
# O motivo é que os dados nessas colunas apresentam um caráter ordinal.
cat_attrs_1h = ['bairro']
cat_attrs_OE = ['bairro_group', 'room_type']



In [387]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
# Pipeline para lidar com os valores numericos
# Aqui, preferi usar o StandardScaler
# Acredito que os dados não estão com escalas tão diferentes a ponto de ser necessário o uso de min-max
# O std scaler pode se sair bem, já que o problema que estamos enfrentando é de regressão linear e terá o uso de gradiente.
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value=0)),
    ('std_scaler', StandardScaler()),
])

In [388]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attrs),
    ("cat_1h", OneHotEncoder(handle_unknown='ignore'), cat_attrs_1h),
    ("cat_OE", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_attrs_OE)
])

housing_prep = full_pipeline.fit_transform(housing)
housing_prep

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 357908 stored elements and shape (34225, 228)>

In [389]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation: ", scores.std())

In [390]:
# from sklearn.linear_model import LinearRegression
# lin_reg = LinearRegression()
# lin_reg.fit(housing_prep, housing_labels)

In [391]:
# from sklearn.tree import DecisionTreeRegressor

# tree_reg = DecisionTreeRegressor()
# tree_reg.fit(housing_prep, housing_labels)

In [392]:
# # Avaliação dos modelos
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(tree_reg, housing_prep, housing_labels, scoring="neg_mean_squared_error", cv=10)
# tree_rmse_scores = np.sqrt(-scores)

In [393]:
# display_scores(tree_rmse_scores)

In [394]:
# lin_scores = cross_val_score(lin_reg, housing_prep, housing_labels, scoring="neg_mean_squared_error", cv=10)
# lin_rmse_scores = np.sqrt(-lin_scores)
# display_scores(lin_rmse_scores)

In [395]:
# from sklearn.ensemble import RandomForestRegressor

# forest_reg = RandomForestRegressor()
# forest_reg.fit(housing_prep, housing_labels)



In [396]:
# forest_reg_scores = cross_val_score(forest_reg, housing_prep, housing_labels, scoring="neg_mean_squared_error", cv=10)
# forestreg_rmse_scores = np.sqrt(-forest_reg_scores)
# display_scores(forestreg_rmse_scores)

**Entre os 3 modelos, o que mostrou um melhor desempenho foi o LinearRegression**

**Porém, como visto no EDA, temos alguns outliers no conjunto de dados**

**Irei fazer o mesmo procedimento, porém agora com a remoção desses outliers**

**Percebi que 

In [397]:
len(housing)

34225

In [398]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(pd.DataFrame(housing_labels))
housing = housing[outlier_pred == 1]
# O mesmo para housing_labels, já que o conjunto dos targets deve ter o mesmo tamanho
housing_labels = housing_labels[outlier_pred == 1]

print(len(housing), len(housing_labels))



28756 28756


**De fato, o IsolationForest indentificou alguns possíveis outliers (até mesmo mais do que eu esperava).**  
**Vamos aplicar o mesmo Pipeline ao novo conjunto de dados.**


In [399]:
housing_prep_no_outliers = full_pipeline.fit_transform(housing)

In [400]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
lin_alg = LinearRegression()
lin_alg.fit(housing_prep_no_outliers, housing_labels)
lin_alg_scores = cross_val_score(lin_alg, housing_prep_no_outliers, housing_labels, scoring="neg_mean_squared_error", cv=10)
lim_rmse_scores = np.sqrt(-lin_alg_scores)
display_scores(lim_rmse_scores)

Scores:  [36.72890967 37.79939105 38.25469467 38.20308932 39.14942564 37.49635664
 38.34036201 37.16606663 38.85025414 37.451084  ]
Mean:  37.94396337633263
Standard deviation:  0.7170074027093468


**Já temos um resultado melhor que todos os outros!**  
**Porém, o erro continua alto, visto que a maioria dos preços se concentram em < 175, o erro ainda é alto** 
**Os outliers estavam dificultando o aprendizado do modelo**

In [401]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prep_no_outliers, housing_labels)
tree_reg_scores = cross_val_score(tree_reg, housing_prep_no_outliers, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_reg_scores)
display_scores(tree_rmse_scores)

Scores:  [48.1609314  50.60356158 49.83688623 50.01664188 50.64844471 48.56795921
 49.34439751 48.87662196 49.94987749 50.30803549]
Mean:  49.6313357448374
Standard deviation:  0.8169835973525131


In [402]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_alg.predict(housing_prep_no_outliers)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

np.float64(37.57898532572028)

**O score no conjunto de treinamento é menor do que nos conjuntos de validação. O modelo ainda está se sobreajustando ao conjunto de treinamento**
**Não testarei o RandomForestRegressor**  
**O resultado da Regressão Linear me parece satisfatória** 

In [403]:
# Alguns testes de previsão
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)



In [404]:
print("Predições: ", lin_alg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

Predições:  [162.14816287  66.72905686  88.06696609  67.7153882   53.92218929]
Labels:  [200, 69, 115, 39, 125]


### **Aperfeiçoamento do modelo**

In [405]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False],
    'n_jobs': [None, -1]
}
lin_reg = LinearRegression()
grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prep_no_outliers.toarray(), housing_labels)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False], 'n_jobs': [None, -1],
                         'positive': [True, False]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [406]:
grid_search.best_params_

{'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [407]:
grid_search.best_estimator_.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [408]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

5564269040380.574 {'fit_intercept': True, 'n_jobs': None, 'positive': True}
37.98789780736308 {'fit_intercept': True, 'n_jobs': None, 'positive': False}
5564269040380.574 {'fit_intercept': True, 'n_jobs': -1, 'positive': True}
37.98789780736308 {'fit_intercept': True, 'n_jobs': -1, 'positive': False}
47.857992996743675 {'fit_intercept': False, 'n_jobs': None, 'positive': True}
38.11608572405034 {'fit_intercept': False, 'n_jobs': None, 'positive': False}
47.857992996743675 {'fit_intercept': False, 'n_jobs': -1, 'positive': True}
38.11608572405034 {'fit_intercept': False, 'n_jobs': -1, 'positive': False}


In [409]:
final_model = grid_search.best_estimator_
predictions = final_model.predict(housing_prep_no_outliers)
final_model_scores = cross_val_score(final_model, housing_prep_no_outliers, housing_labels, scoring="neg_mean_squared_error", cv=10)
final_model_rmse_scores = np.sqrt(-final_model_scores)
display_scores(final_model_rmse_scores)

Scores:  [36.72890967 37.79939105 38.25469467 38.20308932 39.14942564 37.49635664
 38.34036201 37.16606663 38.85025414 37.451084  ]
Mean:  37.94396337633263
Standard deviation:  0.7170074027093468


**No fim, o melhor modelo é o inicial**  
**Vamos testar no conjunto de teste!**

In [431]:
X_test = test_set.drop('price', axis=1)
Y_test = test_set['price'].copy()
X_test = custom_preprocess(X_test)

In [432]:
from sklearn.ensemble import IsolationForest
isolation_forest_final = IsolationForest(random_state=42)
outlier_pred_final_model = isolation_forest_final.fit_predict(pd.DataFrame(Y_test))

In [433]:
X_test = X_test[outlier_pred_final_model == 1]
Y_test = Y_test[outlier_pred_final_model == 1]
X_test_prepared = full_pipeline.transform(X_test)



In [435]:
final_predictions = final_model.predict(X_test_prepared)

In [436]:
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse


np.float64(39.02888844591703)

In [438]:
import joblib
joblib.dump(final_model, "linear_regression_model.pkl")

['linear_regression_model.pkl']

**Nosso modelo final tem desempenho semelhante na validação e teste**   
**Isso indica uma boa generalização do modelo, não está subajustado ou sobreajustado**